# 🚗 Car License Plate Recognition System
## A Comprehensive Analysis and Implementation Guide

---

### 📋 Table of Contents
1. [Project Overview](#project-overview)
2. [System Architecture](#system-architecture)
3. [Data Preparation](#data-preparation)
4. [Model Training](#model-training)
5. [Inference and Testing](#inference-and-testing)
6. [Real-time Detection](#real-time-detection)
7. [Results and Performance](#results-and-performance)
8. [Future Improvements](#future-improvements)

---

## 🎯 Project Overview

This project implements an **end-to-end car license plate detection and recognition system** using state-of-the-art computer vision techniques. The system is specifically designed for **Yemeni license plates** and leverages YOLOv8 (You Only Look Once) for object detection.

### Key Features:
- ✅ **Real-time license plate detection** from images and video streams
- ✅ **YOLOv8-based object detection** with high accuracy
- ✅ **Automated data preprocessing** and annotation conversion
- ✅ **Live streaming support** for real-time monitoring
- ✅ **Professional-grade implementation** with proper error handling

### Technology Stack:
- **Deep Learning**: YOLOv8 (Ultralytics)
- **Computer Vision**: OpenCV, PIL
- **Data Processing**: Pandas, NumPy
- **Streaming**: Real-time video processing
- **Deployment**: Python with modular architecture


## 🏗️ System Architecture

The car plate recognition system follows a modular architecture designed for scalability and maintainability:

```mermaid
graph TD
    A[Input Images/Video] --> B[Data Preprocessing]
    B --> C[YOLOv8 Model Training]
    C --> D[Model Validation]
    D --> E[Inference Engine]
    E --> F[Real-time Detection]
    F --> G[Output Results]
    
    H[Live Stream] --> I[Frame Processing]
    I --> E
    
    J[CSV Annotations] --> K[YOLO Format Conversion]
    K --> B
```

### Core Components:

1. **Data Pipeline** (`convert_to_yolo.py`)
   - Converts CSV annotations to YOLO format
   - Handles multiple datasets (train/validation/test)
   - Ensures proper coordinate normalization

2. **Model Training** (`main.py`, `yemen car plate number model tran.py`)
   - YOLOv8 model initialization and training
   - Configurable hyperparameters
   - GPU acceleration support

3. **Inference Engine** (`model_test.py`, `seperated-plates.py`)
   - Batch processing of test images
   - Plate cropping and extraction
   - Results visualization

4. **Real-time Processing** (`recognition-with-live-stream/live-stream.py`)
   - Live video stream processing
   - Real-time detection and visualization
   - Error handling and reconnection logic


In [ ]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display project structure
print("📁 Project Structure:")
print("=" * 50)
for root, dirs, files in os.walk("."):
    level = root.replace(".", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")


## 📊 Data Preparation

The data preparation phase is crucial for training an effective YOLO model. Our system handles two main datasets:

### Dataset Information:
- **Primary Dataset**: Yemen Car Plate Dataset (v1i.yolov8)
- **Secondary Dataset**: Yemen Plate Dataset
- **Format**: YOLO format with normalized coordinates
- **Classes**: 1 class (License Plate)
- **Total Images**: ~200+ images across train/validation/test splits

### Data Distribution:


In [ ]:
# Analyze dataset distribution
def analyze_dataset_structure():
    """Analyze the structure and distribution of our datasets"""
    
    datasets = {
        "Yemen Car Plate v1i": "src/data/plat number car yemen.v1i.yolov8",
        "Yemen Plate": "src/data/yemen-plate"
    }
    
    results = {}
    
    for name, path in datasets.items():
        if os.path.exists(path):
            train_images = len([f for f in os.listdir(f"{path}/train/images") if f.endswith(('.jpg', '.jpeg', '.png'))])
            val_images = len([f for f in os.listdir(f"{path}/valid/images") if f.endswith(('.jpg', '.jpeg', '.png'))])
            test_images = len([f for f in os.listdir(f"{path}/test/images") if f.endswith(('.jpg', '.jpeg', '.png'))])
            
            results[name] = {
                'train': train_images,
                'validation': val_images,
                'test': test_images,
                'total': train_images + val_images + test_images
            }
    
    return results

# Analyze datasets
dataset_stats = analyze_dataset_structure()

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Dataset comparison
datasets = list(dataset_stats.keys())
train_counts = [dataset_stats[ds]['train'] for ds in datasets]
val_counts = [dataset_stats[ds]['validation'] for ds in datasets]
test_counts = [dataset_stats[ds]['test'] for ds in datasets]

x = np.arange(len(datasets))
width = 0.25

ax1.bar(x - width, train_counts, width, label='Train', alpha=0.8)
ax1.bar(x, val_counts, width, label='Validation', alpha=0.8)
ax1.bar(x + width, test_counts, width, label='Test', alpha=0.8)

ax1.set_xlabel('Dataset')
ax1.set_ylabel('Number of Images')
ax1.set_title('Dataset Distribution Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(datasets, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Pie chart for primary dataset
primary_dataset = "Yemen Car Plate v1i"
if primary_dataset in dataset_stats:
    sizes = [dataset_stats[primary_dataset]['train'], 
             dataset_stats[primary_dataset]['validation'], 
             dataset_stats[primary_dataset]['test']]
    labels = ['Train', 'Validation', 'Test']
    colors = ['#ff9999', '#66b3ff', '#99ff99']
    
    ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax2.set_title(f'{primary_dataset} Split Distribution')

plt.tight_layout()
plt.show()

# Print detailed statistics
print("📈 Dataset Statistics:")
print("=" * 50)
for name, stats in dataset_stats.items():
    print(f"\n{name}:")
    print(f"  Train: {stats['train']} images")
    print(f"  Validation: {stats['validation']} images") 
    print(f"  Test: {stats['test']} images")
    print(f"  Total: {stats['total']} images")
